In [1]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:95% !important;}
div.cell.code_cell.rendered{width:100%;}
div.CodeMirror {font-family:Consolas; font-size:15pt;}
div.output {font-size:15pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:15pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:15px;}
</style>
"""))

# <span style="color:red"> Step.3_PDF (과실비율인정기준) 데이터 정제 </span>

In [3]:
# ========================================
# PDF 참조형 TXT 파일 정제기
# 원본 PDF와 비교하며 안전하게 정제
# ========================================

import pandas as pd
import numpy as np
import re
import unicodedata
from pathlib import Path
import os
from datetime import datetime
import pdfplumber

class PDFReferenceTxtCleaner:
    def __init__(self, txt_path, pdf_path):
        """
        PDF 참조형 TXT 파일 정제기 초기화
        
        Args:
            txt_path: 정제할 TXT 파일 경로
            pdf_path: 참조용 원본 PDF 파일 경로
        """
        self.txt_path = txt_path
        self.pdf_path = pdf_path
        self.original_text = ""
        self.cleaned_text = ""
        self.pdf_pages = []
        self.page_texts = {}
        
        self.correction_stats = {
            'unicode_fixes': 0,
            'korean_fixes': 0,
            'spacing_fixes': 0,
            'legal_term_fixes': 0,
            'code_fixes': 0,
            'page_number_removes': 0,
            'pdf_verified_fixes': 0
        }
        
        # 확실한 교정만 적용하는 보수적 사전
        self.safe_corrections = {
            # 명백한 유니코드 깨짐
            'ᄄᅠᆫ': '',
            'ᄄᅠ': '',
            'ᆨ': '',
            'ᆯ': '',
            'ᇂ': '',
            
            # 확실한 OCR 오류
            '대볍원': '대법원',
            '고등볍원': '고등법원',
            '지방볍원': '지방법원',
            '판걸': '판결',
            '선곡': '선고',
        }
        
        # 의심스러운 패턴 (PDF 확인 필요)
        self.suspicious_patterns = {
            r'대[볍팝펍]원': '대법원',
            r'과실비[놀율]': '과실비율', 
            r'기본과실비[놀율]': '기본과실비율',
            r'수정요[서쇼]': '수정요소',
            r'교차[놀로]': '교차로',
            r'횡단보[놀도되]': '횡단보도',
            r'보행[쟈자]': '보행자',
            r'자동[찻차]': '자동차',
            r'\[보[Il1o0]\]': '[보1]',
            r'\[차[Il1o0]\]': '[차1]',
            r'\[이[Il1o0]\]': '[이1]',
        }

    def load_files(self):
        """TXT 파일과 PDF 파일 로드"""
        # TXT 파일 로드
        try:
            with open(self.txt_path, 'r', encoding='utf-8-sig') as f:
                self.original_text = f.read()
            print(f"✅ TXT 파일 로드 완료: {os.path.basename(self.txt_path)}")
            print(f"📊 TXT 파일 크기: {len(self.original_text):,} 문자")
        except Exception as e:
            print(f"❌ TXT 파일 로드 오류: {e}")
            return False
        
        # PDF 파일 로드
        try:
            with pdfplumber.open(self.pdf_path) as pdf:
                print(f"📖 PDF 파일 로드 중... (총 {len(pdf.pages)}페이지)")
                for i, page in enumerate(pdf.pages):
                    page_text = page.extract_text() or ""
                    self.page_texts[i+1] = page_text
                    if (i+1) % 10 == 0:
                        print(f"   📄 {i+1}페이지 완료...")
                
            print(f"✅ PDF 파일 로드 완료: {os.path.basename(self.pdf_path)}")
            print(f"📊 PDF 총 페이지: {len(self.page_texts)}")
            return True
            
        except Exception as e:
            print(f"❌ PDF 파일 로드 오류: {e}")
            print("⚠️  PDF 없이 안전한 정제만 진행합니다.")
            return False

    def find_text_in_pdf(self, search_text, context_length=30):
        """PDF에서 특정 텍스트 찾기"""
        results = []
        for page_num, page_text in self.page_texts.items():
            if search_text in page_text:
                # 컨텍스트와 함께 위치 찾기
                index = page_text.find(search_text)
                start = max(0, index - context_length)
                end = min(len(page_text), index + len(search_text) + context_length)
                context = page_text[start:end]
                
                results.append({
                    'page': page_num,
                    'context': context,
                    'position': index
                })
        return results

    def verify_correction_with_pdf(self, original_text, proposed_correction):
        """PDF와 비교하여 교정의 정확성 확인"""
        if not self.page_texts:
            return False, "PDF를 참조할 수 없습니다"
        
        # PDF에서 제안된 교정이 실제로 존재하는지 확인
        pdf_matches = self.find_text_in_pdf(proposed_correction)
        
        if pdf_matches:
            return True, f"PDF {len(pdf_matches)}개 페이지에서 확인됨"
        
        # 원본 텍스트가 PDF에 있는지도 확인
        original_matches = self.find_text_in_pdf(original_text)
        if original_matches:
            return False, f"원본이 PDF에서 정확함 (페이지 {original_matches[0]['page']})"
        
        return False, "PDF에서 확인 불가"

    def analyze_with_pdf_reference(self):
        """PDF 참조하여 파일 분석"""
        if not self.original_text:
            print("❌ TXT 파일이 로드되지 않았습니다.")
            return
        
        print("\n🔍 PDF 참조 분석 결과:")
        print("="*60)
        
        # 기본 통계
        lines = self.original_text.split('\n')
        print(f"📄 TXT 총 줄 수: {len(lines):,}")
        print(f"📝 TXT 총 문자 수: {len(self.original_text):,}")
        print(f"🇰🇷 한글 문자 수: {len(re.findall(r'[가-힣]', self.original_text)):,}")
        print(f"📖 PDF 총 페이지: {len(self.page_texts)}")
        
        print(f"\n🚨 발견된 문제점:")
        
        # 1. 확실한 깨짐 (자동 수정 가능)
        certain_issues = []
        for wrong, correct in self.safe_corrections.items():
            count = self.original_text.count(wrong)
            if count > 0:
                certain_issues.append(f"{wrong} → {correct} ({count}회)")
        
        if certain_issues:
            print(f"\n✅ 확실한 오류 (자동 수정 가능):")
            for issue in certain_issues:
                print(f"   - {issue}")
        
        # 2. 의심스러운 패턴 (PDF 확인 필요)
        suspicious_issues = []
        for pattern, correct in self.suspicious_patterns.items():
            matches = re.findall(pattern, self.original_text)
            if matches:
                suspicious_issues.append({
                    'pattern': pattern,
                    'matches': matches,
                    'correction': correct,
                    'count': len(matches)
                })
        
        if suspicious_issues:
            print(f"\n⚠️  의심스러운 패턴 (PDF 확인 필요):")
            for issue in suspicious_issues[:5]:  # 상위 5개만
                print(f"   - {issue['pattern']} → {issue['correction']} ({issue['count']}회)")
                print(f"     발견된 예: {issue['matches'][:3]}")
        
        print("="*60)

    def clean_safe_corrections(self, text):
        """확실한 교정만 적용"""
        for wrong, correct in self.safe_corrections.items():
            if wrong in text:
                count = text.count(wrong)
                text = text.replace(wrong, correct)
                self.correction_stats['unicode_fixes'] += count
                print(f"   ✅ {wrong} → {correct} ({count}회 수정)")
        
        return text

    def clean_with_pdf_verification(self, text):
        """PDF 검증과 함께 의심스러운 패턴 정제"""
        if not self.page_texts:
            print("   ⚠️  PDF 참조 불가, 의심스러운 패턴은 건너뜀")
            return text
        
        verified_corrections = 0
        
        for pattern, suggested_correction in self.suspicious_patterns.items():
            matches = list(re.finditer(pattern, text))
            
            for match in matches:
                original_match = match.group(0)
                
                # PDF에서 확인
                is_correct, reason = self.verify_correction_with_pdf(
                    original_match, suggested_correction
                )
                
                if is_correct:
                    text = text.replace(original_match, suggested_correction, 1)
                    verified_corrections += 1
                    print(f"   ✅ PDF 확인: {original_match} → {suggested_correction}")
                else:
                    print(f"   ❌ PDF 미확인: {original_match} (건너뜀) - {reason}")
        
        self.correction_stats['pdf_verified_fixes'] = verified_corrections
        return text

    def interactive_correction_review(self, text):
        """사용자와 상호작용하며 의심스러운 부분 검토"""
        print(f"\n🤔 의심스러운 패턴 검토 (사용자 확인 필요):")
        print("="*50)
        
        user_corrections = 0
        
        for pattern, suggested_correction in self.suspicious_patterns.items():
            matches = list(re.finditer(pattern, text))
            
            if not matches:
                continue
            
            print(f"\n패턴: {pattern} → {suggested_correction}")
            print(f"발견된 수: {len(matches)}개")
            
            for i, match in enumerate(matches[:3]):  # 최대 3개만 검토
                original_match = match.group(0)
                start, end = match.span()
                
                # 컨텍스트 표시
                context_start = max(0, start - 30)
                context_end = min(len(text), end + 30)
                context = text[context_start:context_end]
                
                print(f"\n예시 {i+1}: ...{context}...")
                print(f"'{original_match}' → '{suggested_correction}'로 바꾸시겠습니까?")
                
                # PDF에서 찾은 정보 표시
                pdf_results = self.find_text_in_pdf(suggested_correction)
                if pdf_results:
                    print(f"   📖 PDF에서 '{suggested_correction}' 발견: {len(pdf_results)}회")
                    print(f"      예시: {pdf_results[0]['context'][:50]}...")
                
                user_input = input("   (y)예 / (n)아니오 / (s)건너뛰기: ").strip().lower()
                
                if user_input in ['y', 'yes', '예', 'ㅇ']:
                    text = text.replace(original_match, suggested_correction, 1)
                    user_corrections += 1
                    print(f"   ✅ 수정됨: {original_match} → {suggested_correction}")
                elif user_input in ['s', 'skip', '건너뛰기', 'ㅅ']:
                    break
                else:
                    print(f"   ❌ 건너뜀: {original_match}")
        
        self.correction_stats['korean_fixes'] = user_corrections
        return text

    def clean_formatting(self, text):
        """서식 정리 (안전한 것만)"""
        # 페이지 번호 제거
        page_numbers = re.findall(r'-\s*\d+\s*-', text)
        text = re.sub(r'-\s*\d+\s*-', '', text)
        self.correction_stats['page_number_removes'] += len(page_numbers)
        
        # 과도한 공백 정리
        text = re.sub(r'\n\s*\n\s*\n+', '\n\n', text)
        text = re.sub(r' +', ' ', text)
        text = re.sub(r'\t+', ' ', text)
        
        return text

    def clean_all_with_pdf_reference(self, interactive=True):
        """PDF 참조하여 전체 정제 실행"""
        if not self.original_text:
            print("❌ 파일이 로드되지 않았습니다.")
            return
        
        print("🧹 PDF 참조 텍스트 정제 시작...")
        print("="*50)
        
        self.cleaned_text = self.original_text
        
        print("1️⃣ 확실한 오류 자동 수정 중...")
        self.cleaned_text = self.clean_safe_corrections(self.cleaned_text)
        
        if self.page_texts:
            print("\n2️⃣ PDF 검증을 통한 의심스러운 패턴 정제 중...")
            self.cleaned_text = self.clean_with_pdf_verification(self.cleaned_text)
        
        if interactive:
            print("\n3️⃣ 사용자 확인이 필요한 패턴 검토...")
            self.cleaned_text = self.interactive_correction_review(self.cleaned_text)
        
        print("\n4️⃣ 서식 정리 중...")
        self.cleaned_text = self.clean_formatting(self.cleaned_text)
        
        print("\n✅ 정제 완료!")
        self.print_detailed_stats()

    def print_detailed_stats(self):
        """상세 정제 통계 출력"""
        print(f"\n📊 PDF 참조 정제 결과:")
        print("="*50)
        print(f"🔧 확실한 오류 수정: {self.correction_stats['unicode_fixes']}개")
        print(f"📖 PDF 검증 수정: {self.correction_stats['pdf_verified_fixes']}개")
        print(f"👤 사용자 확인 수정: {self.correction_stats['korean_fixes']}개")
        print(f"🗑️  페이지번호 제거: {self.correction_stats['page_number_removes']}개")
        
        total_fixes = sum(self.correction_stats.values())
        print(f"\n🎯 총 수정사항: {total_fixes}개")
        
        # 파일 크기 변화
        original_size = len(self.original_text)
        cleaned_size = len(self.cleaned_text)
        size_diff = original_size - cleaned_size
        
        print(f"\n📦 파일 크기 변화:")
        print(f"   원본: {original_size:,} 문자")
        print(f"   정제후: {cleaned_size:,} 문자")
        print(f"   차이: {size_diff:,} 문자")

    def compare_with_pdf_samples(self, num_samples=3):
        """PDF와 비교한 정제 샘플 출력"""
        if not self.cleaned_text or not self.page_texts:
            print("❌ 비교할 데이터가 없습니다.")
            return
        
        print(f"\n🔍 PDF 참조 정제 전후 비교:")
        print("="*70)
        
        # 실제 수정된 부분들 찾기
        corrections_made = []
        
        # 확실한 수정사항들
        for wrong, correct in self.safe_corrections.items():
            if wrong in self.original_text and correct in self.cleaned_text:
                for match in re.finditer(re.escape(wrong), self.original_text):
                    start, end = match.span()
                    context_start = max(0, start - 25)
                    context_end = min(len(self.original_text), end + 25)
                    
                    original_context = self.original_text[context_start:context_end]
                    cleaned_context = original_context.replace(wrong, correct)
                    
                    # PDF에서 해당 내용 찾기
                    pdf_results = self.find_text_in_pdf(correct)
                    pdf_info = f"PDF {len(pdf_results)}개 페이지에서 확인" if pdf_results else "PDF 미확인"
                    
                    corrections_made.append({
                        'type': f'자동수정 ({wrong}→{correct})',
                        'original': original_context,
                        'cleaned': cleaned_context,
                        'pdf_verification': pdf_info
                    })
        
        # 상위 샘플 출력
        for i, correction in enumerate(corrections_made[:num_samples]):
            print(f"\n{i+1}. {correction['type']}")
            print(f"   ❌ 원본: {correction['original']}")
            print(f"   ✅ 정제: {correction['cleaned']}")
            print(f"   📖 PDF 검증: {correction['pdf_verification']}")
            print("-" * 50)

    def save_cleaned_file(self, output_path=None):
        """정제된 파일 저장"""
        if not self.cleaned_text:
            print("❌ 정제된 텍스트가 없습니다.")
            return
        
        if not output_path:
            original_path = Path(self.txt_path)
            timestamp = datetime.now().strftime("%Y%m%d_%H%M")
            output_path = original_path.parent / f"{original_path.stem}_cleaned_pdf_verified_{timestamp}.txt"
        
        try:
            with open(output_path, 'w', encoding='utf-8-sig') as f:
                f.write(self.cleaned_text)
            
            print(f"💾 정제된 파일 저장 완료: {output_path}")
            return str(output_path)
        except Exception as e:
            print(f"❌ 파일 저장 오류: {e}")
            return None

# ========================================
# 실행 코드
# ========================================

def main():
    """메인 실행 함수"""
    print("🚀 PDF 참조형 TXT 파일 정제기 시작")
    print("="*60)
    
    # 파일 경로
    txt_path = r"C:\project\2stProject_jun\jun\231107_과실비율인정기준_온라인용_extracted_text.txt"
    pdf_path = r"C:\project\2stProject_jun\jun\과실비율PDF\231107_과실비율인정기준_온라인용.pdf"
    
    # 정제기 초기화
    cleaner = PDFReferenceTxtCleaner(txt_path, pdf_path)
    
    # 파일 로드
    if not cleaner.load_files():
        return
    
    # PDF 참조 분석
    cleaner.analyze_with_pdf_reference()
    
    # 정제 진행 여부 확인
    print(f"\n정제를 진행하시겠습니까?")
    print("- 자동 정제: 확실한 오류만 수정")
    print("- PDF 검증: 원본과 비교하여 의심스러운 부분 확인")
    print("- 사용자 확인: 애매한 부분은 직접 확인")
    
    user_input = input("\n진행하시겠습니까? (y/n): ").strip().lower()
    
    if user_input in ['y', 'yes', '예', 'ㅇ']:
        # 상호작용 모드 선택
        interactive_input = input("사용자 확인 모드를 사용하시겠습니까? (y/n): ").strip().lower()
        interactive = interactive_input in ['y', 'yes', '예', 'ㅇ']
        
        # 정제 실행
        cleaner.clean_all_with_pdf_reference(interactive=interactive)
        
        # PDF 비교 샘플 출력
        cleaner.compare_with_pdf_samples()
        
        # 저장 여부 확인
        save_input = input(f"\n정제된 파일을 저장하시겠습니까? (y/n): ").strip().lower()
        
        if save_input in ['y', 'yes', '예', 'ㅇ']:
            output_path = cleaner.save_cleaned_file()
            if output_path:
                print(f"\n🎉 PDF 참조 정제 완료!")
                print(f"📁 저장된 파일: {output_path}")
                print(f"📖 원본 PDF와 비교 검증을 통해 안전하게 정제되었습니다.")
    else:
        print("정제를 취소했습니다.")

if __name__ == "__main__":
    main()

🚀 PDF 참조형 TXT 파일 정제기 시작
✅ TXT 파일 로드 완료: 231107_과실비율인정기준_온라인용_extracted_text.txt
📊 TXT 파일 크기: 665,696 문자
📖 PDF 파일 로드 중... (총 600페이지)
   📄 10페이지 완료...
   📄 20페이지 완료...
   📄 30페이지 완료...
   📄 40페이지 완료...
   📄 50페이지 완료...
   📄 60페이지 완료...
   📄 70페이지 완료...
   📄 80페이지 완료...
   📄 90페이지 완료...
   📄 100페이지 완료...
   📄 110페이지 완료...
   📄 120페이지 완료...
   📄 130페이지 완료...
   📄 140페이지 완료...
   📄 150페이지 완료...
   📄 160페이지 완료...
   📄 170페이지 완료...
   📄 180페이지 완료...
   📄 190페이지 완료...
   📄 200페이지 완료...
   📄 210페이지 완료...
   📄 220페이지 완료...
   📄 230페이지 완료...
   📄 240페이지 완료...
   📄 250페이지 완료...
   📄 260페이지 완료...
   📄 270페이지 완료...
   📄 280페이지 완료...
   📄 290페이지 완료...
   📄 300페이지 완료...
   📄 310페이지 완료...
   📄 320페이지 완료...
   📄 330페이지 완료...
   📄 340페이지 완료...
   📄 350페이지 완료...
   📄 360페이지 완료...
   📄 370페이지 완료...
   📄 380페이지 완료...
   📄 390페이지 완료...
   📄 400페이지 완료...
   📄 410페이지 완료...
   📄 420페이지 완료...
   📄 430페이지 완료...
   📄 440페이지 완료...
   📄 450페이지 완료...
   📄 460페이지 완료...
   📄 470페이지 완료...
   📄 480페이지 완료...
   📄 490페이지 완

   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율


   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율


   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율


   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율
   ✅ PDF 확인: 과실비율 → 과실비율


...

   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인

   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인

   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인

   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인: 자동차 → 자동차
   ✅ PDF 확인


정제된 파일을 저장하시겠습니까? (y/n): y
💾 정제된 파일 저장 완료: C:\project\2stProject_jun\jun\231107_과실비율인정기준_온라인용_extracted_text_cleaned_pdf_verified_20250808_1424.txt

🎉 PDF 참조 정제 완료!
📁 저장된 파일: C:\project\2stProject_jun\jun\231107_과실비율인정기준_온라인용_extracted_text_cleaned_pdf_verified_20250808_1424.txt
📖 원본 PDF와 비교 검증을 통해 안전하게 정제되었습니다.
